# Skip-gram addr2vec

In this notebook, I'll convert addresses to vectors through TensorFlow and understand the ability to correct for mistyping.


## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Load the [openaddress US North East dataset](https://s3.amazonaws.com/data.openaddresses.io/openaddr-collected-us_northeast.zip), and extract onto 'openaddr' directory if not found. read the csv files and load address dictionaries

In [2]:
import glob
import os
import pandas as pd

from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'openaddr'
dataset_filename = 'openaddr-collected-us_northeast.zip'
dataset_name = 'Openaddress Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/data.openaddresses.io/openaddr-collected-us_northeast.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)


id_to_address = {}
address_to_id = {}
i = 0
for state in os.listdir('./openaddr/us'):
    
    for filename in glob.glob('./openaddr/us/{}/*.csv'.format(state)):
        csv = pd.read_csv(filename)
        if i == 0:
            print("Example data: {}".format(csv.iloc[2]))

        stack = np.stack((csv['CITY'],), axis=-1)
        for j in stack:
            addr = " ".join([str(k).lower()
                             for k in j if not isinstance(k, type(np.nan))])
            if addr not in address_to_id and addr != '' and addr != ' ':
                id_to_address[i] = addr
                address_to_id[addr] = i
                i += 1

del csv

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Example data: LON                 -69.7958
LAT                  44.6535
NUMBER                  1033
STREET             E Pond Rd
UNIT                     NaN
CITY                     NaN
DISTRICT            Somerset
REGION                    ME
POSTCODE                4978
ID                       NaN
HASH        8bbbfb5152ee2563
Name: 2, dtype: object
Example data: LON                   -80.0287
LAT                    40.7328
NUMBER                    1260
STREET      MARS-EVANS CITY RD
UNIT                       NaN
CITY                 ADAMS TWP
DISTRICT                   NaN
REGION                     NaN
POSTCODE                   NaN
ID                         NaN
HASH          1b447375afa847e4
Name: 2, dtype: object


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,4,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler

## Preprocessing

The addresses have all been converted to lower case and only city names are being used and tested for correction cases. vocabulary is the letters from which the city is made up of. We vectorise by counting the number of letters and normalizing to the total number between zero and one. Visualisations below.

In [3]:
print([i for i in address_to_id.keys()][:30])

['adams twp', 'cranberry twp', 'middlesex twp', 'mars boro', 'allegheny twp', 'bruin boro', 'brady twp', 'slippery rock twp', 'buffalo twp', 'winfield twp', 'butler city', 'butler twp', 'penn twp', 'callery boro', 'center twp', 'clay twp', 'cherry twp', 'cherry valley boro', 'chicora boro', 'clearfield twp', 'donegal twp', 'summit twp', 'clinton twp', 'connoq boro', 'forward twp', 'connoq twp', 'lancaster twp', 'seven fields boro', 'fairview twp', 'east butler boro']


In [4]:
print("Total addresses: {}".format(len(address_to_id)))

vocab_to_id = {}
int_to_vocab = {}
idx = 0
for address in address_to_id.keys():
    for j in range(len(address)):
        if address[j] not in vocab_to_id:
            vocab_to_id[address[j]] = idx
            int_to_vocab[idx] = address[j]
            idx += 1

vocab_size = len(vocab_to_id)
print("Total unique letters: {}".format(vocab_size))

Total addresses: 4155
Total unique letters: 45


In [17]:
features = []
labels = []
labels_to_id = {}
count = 0

for address, idx in address_to_id.items():
    X = np.zeros(vocab_size)
    Y = np.zeros(len(address_to_id))
    for j in range(len(address)):
        X[vocab_to_id[address[j]]] += 1 
    
    # normalize input sizes 
    X = ((X - X.min()) / (X.max() - X.min())) + 0.001
    
    features.append(X)
    
    labels_to_id[idx] = count
    Y[idx] = 1
    labels.append(Y)
    count += 1
    
print("Example feature vector and label:")
print(features[500])
print(labels[500])

Example feature vector and label:
[1.001e+00 1.000e-03 1.001e+00 1.001e+00 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.001e+00 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03]
[0. 0. 0. ... 0. 0. 0.]


In [19]:
# Size of the encoding layer (the hidden layer)
encoding_dim = 500
alpha = 0.1

# Input and target placeholders
inp_shape = vocab_size

inputs_ = tf.placeholder(tf.float32 ,(None, inp_shape))
targets_ = tf.placeholder(tf.float32 ,(None, len(address_to_id)))

# Input layer is 45
x1 = tf.expand_dims(inputs_, axis=-1)
x1 = tf.layers.conv1d(x1, filters=32, kernel_size=5, strides=1, padding='valid')
x1 = tf.nn.dropout(x1, 0.5)
relu1 = tf.maximum(alpha * x1, x1)
print(relu1.shape)
# 41x32

x2 = tf.layers.conv1d(relu1, filters=64, kernel_size=5, strides=1, padding='valid')
x2 = tf.nn.dropout(x2, 0.5)
relu2 = tf.maximum(alpha * x2, x2)
print(relu2.shape)
# 37x64

x3 = tf.layers.conv1d(relu2, filters=128, kernel_size=5, strides=1, padding='valid')
x3 = tf.nn.dropout(x3, 0.5)
relu3 = tf.maximum(alpha * x3, x3)
print(relu3.shape)
# 33x128

# global average pooling layer
first_h_dim = tf.reduce_mean(relu3, axis=2)
print(first_h_dim.shape)

# dense layer
encoded = tf.layers.dense(first_h_dim, encoding_dim)
relu4 = tf.maximum(alpha * encoded, encoded)
print(relu4.shape)

# Output layer logits
logits = tf.layers.dense(relu4, len(address_to_id), activation=None)
print(logits.shape)
# Sigmoid output from logits
#decoded = tf.nn.sigmoid(logits, name='outputs')

# loss = tf.log(tf.reduce_sum(tf.abs(targets_ - logits)) + 1)
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=targets_, logits=logits)
# Mean of the loss
cost = tf.reduce_mean(loss)




(?, 41, 32)
(?, 37, 64)
(?, 33, 128)
(?, 33)
(?, 500)
(?, 4155)


In [20]:
# Adam optimizer
opt = tf.train.AdamOptimizer(0.0005).minimize(cost)

# Create the session
sess = tf.Session()

epochs = 500
batch_size = 128
sess.run(tf.global_variables_initializer())
for e in range(epochs):
    for ii in range((len(features)//batch_size) - 1):
        batch = features[ii*batch_size:(ii + 1)*batch_size]
        
        # add random noise
        for jj in batch:
            jj[np.random.randint(vocab_size)] += 0.01
            jj[np.random.randint(vocab_size)] -= 0.01
        
        targ = np.array(labels[ii*batch_size:(ii + 1)*batch_size])

        feed = {inputs_: batch, targets_: targ}
        batch_cost, _ = sess.run([cost, opt], feed_dict=feed)

        print("Epoch: {}/{}...".format(e+1, epochs),
              "Training loss: {:.4f}".format(batch_cost))

Epoch: 1/500... Training loss: 8.3324
Epoch: 1/500... Training loss: 8.3323
Epoch: 1/500... Training loss: 8.3321
Epoch: 1/500... Training loss: 8.3325
Epoch: 1/500... Training loss: 8.3323
Epoch: 1/500... Training loss: 8.3330
Epoch: 1/500... Training loss: 8.3328
Epoch: 1/500... Training loss: 8.3330
Epoch: 1/500... Training loss: 8.3335
Epoch: 1/500... Training loss: 8.3336
Epoch: 1/500... Training loss: 8.3339
Epoch: 1/500... Training loss: 8.3342
Epoch: 1/500... Training loss: 8.3343
Epoch: 1/500... Training loss: 8.3346
Epoch: 1/500... Training loss: 8.3348
Epoch: 1/500... Training loss: 8.3349
Epoch: 1/500... Training loss: 8.3351
Epoch: 1/500... Training loss: 8.3353
Epoch: 1/500... Training loss: 8.3353
Epoch: 1/500... Training loss: 8.3357
Epoch: 1/500... Training loss: 8.3357
Epoch: 1/500... Training loss: 8.3361
Epoch: 1/500... Training loss: 8.3363
Epoch: 1/500... Training loss: 8.3366
Epoch: 1/500... Training loss: 8.3368
Epoch: 1/500... Training loss: 8.3371
Epoch: 1/500

Epoch: 8/500... Training loss: 8.3302
Epoch: 8/500... Training loss: 8.3417
Epoch: 8/500... Training loss: 8.3414
Epoch: 8/500... Training loss: 8.3312
Epoch: 8/500... Training loss: 8.3270
Epoch: 8/500... Training loss: 8.3163
Epoch: 8/500... Training loss: 8.3168
Epoch: 8/500... Training loss: 8.3115
Epoch: 8/500... Training loss: 8.3099
Epoch: 8/500... Training loss: 8.3037
Epoch: 8/500... Training loss: 8.3072
Epoch: 8/500... Training loss: 8.2985
Epoch: 8/500... Training loss: 8.2944
Epoch: 8/500... Training loss: 8.2838
Epoch: 8/500... Training loss: 8.2695
Epoch: 8/500... Training loss: 8.2526
Epoch: 8/500... Training loss: 8.2583
Epoch: 8/500... Training loss: 8.2900
Epoch: 8/500... Training loss: 8.3243
Epoch: 9/500... Training loss: 8.0112
Epoch: 9/500... Training loss: 8.0109
Epoch: 9/500... Training loss: 7.9837
Epoch: 9/500... Training loss: 8.0848
Epoch: 9/500... Training loss: 8.0119
Epoch: 9/500... Training loss: 8.0277
Epoch: 9/500... Training loss: 7.9538
Epoch: 9/500

Epoch: 15/500... Training loss: 3.0373
Epoch: 15/500... Training loss: 2.9587
Epoch: 15/500... Training loss: 2.7458
Epoch: 15/500... Training loss: 2.8832
Epoch: 15/500... Training loss: 2.8755
Epoch: 15/500... Training loss: 2.7218
Epoch: 15/500... Training loss: 2.8112
Epoch: 15/500... Training loss: 2.8775
Epoch: 15/500... Training loss: 2.7382
Epoch: 15/500... Training loss: 2.6657
Epoch: 15/500... Training loss: 2.2605
Epoch: 16/500... Training loss: 2.5744
Epoch: 16/500... Training loss: 2.5878
Epoch: 16/500... Training loss: 2.6117
Epoch: 16/500... Training loss: 2.8054
Epoch: 16/500... Training loss: 2.6120
Epoch: 16/500... Training loss: 2.7058
Epoch: 16/500... Training loss: 2.7597
Epoch: 16/500... Training loss: 2.8462
Epoch: 16/500... Training loss: 2.9349
Epoch: 16/500... Training loss: 3.3038
Epoch: 16/500... Training loss: 3.3097
Epoch: 16/500... Training loss: 3.2784
Epoch: 16/500... Training loss: 2.9350
Epoch: 16/500... Training loss: 2.9065
Epoch: 16/500... Training

Epoch: 22/500... Training loss: 1.2024
Epoch: 22/500... Training loss: 1.4375
Epoch: 22/500... Training loss: 1.3030
Epoch: 22/500... Training loss: 1.1199
Epoch: 22/500... Training loss: 1.3036
Epoch: 22/500... Training loss: 1.3671
Epoch: 22/500... Training loss: 1.1483
Epoch: 22/500... Training loss: 1.0192
Epoch: 22/500... Training loss: 1.0967
Epoch: 22/500... Training loss: 1.2195
Epoch: 22/500... Training loss: 1.3186
Epoch: 22/500... Training loss: 1.2749
Epoch: 22/500... Training loss: 1.2862
Epoch: 22/500... Training loss: 1.2299
Epoch: 22/500... Training loss: 1.0439
Epoch: 23/500... Training loss: 1.5149
Epoch: 23/500... Training loss: 1.4759
Epoch: 23/500... Training loss: 1.2381
Epoch: 23/500... Training loss: 1.4693
Epoch: 23/500... Training loss: 1.1674
Epoch: 23/500... Training loss: 1.4562
Epoch: 23/500... Training loss: 1.2491
Epoch: 23/500... Training loss: 1.3467
Epoch: 23/500... Training loss: 1.2343
Epoch: 23/500... Training loss: 1.2617
Epoch: 23/500... Training

Epoch: 29/500... Training loss: 0.9495
Epoch: 29/500... Training loss: 1.0062
Epoch: 29/500... Training loss: 0.7001
Epoch: 29/500... Training loss: 0.9434
Epoch: 29/500... Training loss: 0.6461
Epoch: 29/500... Training loss: 0.7038
Epoch: 29/500... Training loss: 0.7024
Epoch: 29/500... Training loss: 0.9373
Epoch: 29/500... Training loss: 0.8745
Epoch: 29/500... Training loss: 0.7296
Epoch: 29/500... Training loss: 0.9070
Epoch: 29/500... Training loss: 0.7675
Epoch: 29/500... Training loss: 0.7696
Epoch: 29/500... Training loss: 0.8171
Epoch: 29/500... Training loss: 0.6231
Epoch: 29/500... Training loss: 0.6974
Epoch: 29/500... Training loss: 0.9709
Epoch: 29/500... Training loss: 0.6769
Epoch: 29/500... Training loss: 0.7637
Epoch: 29/500... Training loss: 0.6099
Epoch: 30/500... Training loss: 0.9511
Epoch: 30/500... Training loss: 0.8425
Epoch: 30/500... Training loss: 0.7748
Epoch: 30/500... Training loss: 0.9658
Epoch: 30/500... Training loss: 0.8241
Epoch: 30/500... Training

...

Epoch: 469/500... Training loss: 0.0025
Epoch: 469/500... Training loss: 0.0050
Epoch: 469/500... Training loss: 0.0265
Epoch: 469/500... Training loss: 0.0018
Epoch: 469/500... Training loss: 0.0038
Epoch: 469/500... Training loss: 0.0019
Epoch: 469/500... Training loss: 0.0029
Epoch: 469/500... Training loss: 0.0061
Epoch: 469/500... Training loss: 0.0004
Epoch: 469/500... Training loss: 0.0065
Epoch: 469/500... Training loss: 0.0008
Epoch: 469/500... Training loss: 0.0025
Epoch: 469/500... Training loss: 0.0016
Epoch: 469/500... Training loss: 0.0441
Epoch: 469/500... Training loss: 0.0018
Epoch: 469/500... Training loss: 0.0019
Epoch: 469/500... Training loss: 0.0057
Epoch: 469/500... Training loss: 0.0036
Epoch: 469/500... Training loss: 0.0041
Epoch: 469/500... Training loss: 0.0009
Epoch: 469/500... Training loss: 0.0146
Epoch: 469/500... Training loss: 0.0019
Epoch: 469/500... Training loss: 0.0009
Epoch: 469/500... Training loss: 0.0007
Epoch: 469/500... Training loss: 0.0461


Epoch: 476/500... Training loss: 0.0053
Epoch: 476/500... Training loss: 0.0165
Epoch: 476/500... Training loss: 0.0035
Epoch: 476/500... Training loss: 0.0296
Epoch: 476/500... Training loss: 0.0194
Epoch: 476/500... Training loss: 0.0040
Epoch: 476/500... Training loss: 0.0369
Epoch: 476/500... Training loss: 0.0016
Epoch: 476/500... Training loss: 0.0047
Epoch: 476/500... Training loss: 0.0188
Epoch: 476/500... Training loss: 0.0030
Epoch: 476/500... Training loss: 0.0152
Epoch: 476/500... Training loss: 0.0022
Epoch: 476/500... Training loss: 0.0006
Epoch: 476/500... Training loss: 0.0012
Epoch: 476/500... Training loss: 0.0360
Epoch: 476/500... Training loss: 0.0146
Epoch: 476/500... Training loss: 0.0019
Epoch: 476/500... Training loss: 0.0031
Epoch: 476/500... Training loss: 0.0006
Epoch: 476/500... Training loss: 0.0005
Epoch: 476/500... Training loss: 0.0509
Epoch: 476/500... Training loss: 0.0011
Epoch: 476/500... Training loss: 0.0011
Epoch: 476/500... Training loss: 0.0009


Epoch: 482/500... Training loss: 0.0039
Epoch: 482/500... Training loss: 0.1234
Epoch: 482/500... Training loss: 0.0137
Epoch: 482/500... Training loss: 0.0024
Epoch: 482/500... Training loss: 0.0010
Epoch: 482/500... Training loss: 0.0050
Epoch: 483/500... Training loss: 0.0034
Epoch: 483/500... Training loss: 0.0031
Epoch: 483/500... Training loss: 0.0018
Epoch: 483/500... Training loss: 0.0014
Epoch: 483/500... Training loss: 0.0057
Epoch: 483/500... Training loss: 0.0058
Epoch: 483/500... Training loss: 0.0010
Epoch: 483/500... Training loss: 0.0031
Epoch: 483/500... Training loss: 0.0002
Epoch: 483/500... Training loss: 0.0012
Epoch: 483/500... Training loss: 0.0066
Epoch: 483/500... Training loss: 0.0112
Epoch: 483/500... Training loss: 0.0205
Epoch: 483/500... Training loss: 0.0236
Epoch: 483/500... Training loss: 0.0271
Epoch: 483/500... Training loss: 0.0446
Epoch: 483/500... Training loss: 0.0598
Epoch: 483/500... Training loss: 0.0043
Epoch: 483/500... Training loss: 0.0014


Epoch: 489/500... Training loss: 0.0029
Epoch: 489/500... Training loss: 0.0380
Epoch: 489/500... Training loss: 0.0012
Epoch: 489/500... Training loss: 0.0107
Epoch: 489/500... Training loss: 0.0160
Epoch: 489/500... Training loss: 0.0050
Epoch: 489/500... Training loss: 0.0037
Epoch: 489/500... Training loss: 0.0369
Epoch: 489/500... Training loss: 0.0015
Epoch: 489/500... Training loss: 0.0018
Epoch: 489/500... Training loss: 0.0025
Epoch: 489/500... Training loss: 0.0267
Epoch: 489/500... Training loss: 0.0010
Epoch: 489/500... Training loss: 0.0079
Epoch: 489/500... Training loss: 0.0021
Epoch: 489/500... Training loss: 0.0316
Epoch: 490/500... Training loss: 0.0016
Epoch: 490/500... Training loss: 0.0019
Epoch: 490/500... Training loss: 0.0022
Epoch: 490/500... Training loss: 0.0012
Epoch: 490/500... Training loss: 0.0011
Epoch: 490/500... Training loss: 0.0283
Epoch: 490/500... Training loss: 0.0006
Epoch: 490/500... Training loss: 0.0018
Epoch: 490/500... Training loss: 0.0015


Epoch: 496/500... Training loss: 0.0439
Epoch: 496/500... Training loss: 0.0084
Epoch: 496/500... Training loss: 0.0007
Epoch: 496/500... Training loss: 0.0665
Epoch: 496/500... Training loss: 0.0057
Epoch: 496/500... Training loss: 0.0056
Epoch: 496/500... Training loss: 0.1370
Epoch: 496/500... Training loss: 0.0154
Epoch: 496/500... Training loss: 0.0582
Epoch: 496/500... Training loss: 0.0200
Epoch: 496/500... Training loss: 0.0183
Epoch: 496/500... Training loss: 0.0020
Epoch: 496/500... Training loss: 0.0045
Epoch: 496/500... Training loss: 0.0530
Epoch: 496/500... Training loss: 0.0730
Epoch: 496/500... Training loss: 0.0015
Epoch: 496/500... Training loss: 0.0015
Epoch: 496/500... Training loss: 0.0317
Epoch: 496/500... Training loss: 0.0009
Epoch: 496/500... Training loss: 0.0008
Epoch: 496/500... Training loss: 0.0025
Epoch: 497/500... Training loss: 0.0408
Epoch: 497/500... Training loss: 0.0040
Epoch: 497/500... Training loss: 0.0016
Epoch: 497/500... Training loss: 0.0027


In [21]:
# originals
# ['adams twp', 'cranberry twp', 'middlesex twp', 'mars boro', 'allegheny twp', 
# 'bruin boro', 'brady twp', 'slippery rock twp', 'buffalo twp', 'winfield twp', 'butler city', 
# 'butler twp', 'penn twp', 'callery boro', 'center twp', 'clay twp', 'cherry twp', 'cherry valley boro', 
# 'chicora boro', 'clearfield twp', 'donegal twp', 'summit twp', 'clinton twp', 'connoq boro', 'forward twp', 
# 'connoq twp', 'lancaster twp', 'seven fields boro', 'fairview twp', 'east butler boro']


test_words = ['aulola', 'wale', 'orchard park', 'holland', 'boston', 'colden', 
              'concord', 'collins', 'north collins', 'sardinia', 'brant', 'evans', 'eden',
              'hamburg', 'lackawanna', 'west seneca', 'elma', 'marilla', 'cheektowaga', 
              'lancapter', 'aldest', 'grant islands', 'amherst', 'clarence', 'newstead', 
              'tonawanda', 'bufalo', 'goshen ny', 'monroe ny']

in_words = []
for word in test_words:
    X = np.zeros(vocab_size)
    for j in range(len(word)):
        if word[j] in vocab_to_id:
            X[vocab_to_id[word[j]]] += 1
            
    # normalize input sizes 
    X = ((X - X.min()) / (X.max() - X.min())) + 0.001
    in_words.append(X)
    
in_words = np.array(in_words)

reconstructed, compressed = sess.run([logits, encoded], feed_dict={inputs_: in_words})

out_words = []
for out, inp in zip(reconstructed, test_words):
    best_match = np.argmax(out)
    
    out_words.append((id_to_address[labels_to_id[best_match]], inp))
    
        
print(out_words)

[('dallas', 'aulola'), ('wel', 'wale'), ('orchard park', 'orchard park'), ('holland', 'holland'), ('boonton', 'boston'), ('delanco', 'colden'), ('crompond', 'concord'), ('collins', 'collins'), ('north collins', 'north collins'), ('indiana', 'sardinia'), ('brant', 'brant'), ('evans', 'evans'), ('eden', 'eden'), ('hamburg', 'hamburg'), ('lackawanna', 'lackawanna'), ('west seneca', 'west seneca'), ('elma', 'elma'), ('marilla', 'marilla'), ('cheektowaga', 'cheektowaga'), ('canal campsites', 'lancapter'), ('seattle', 'aldest'), ('north arlington', 'grant islands'), ('amherst', 'amherst'), ('ceres', 'clarence'), ('newstead', 'newstead'), ('adamstown', 'tonawanda'), ('blakely borough', 'bufalo'), ('basking ridge', 'goshen ny'), ('tyrone bor', 'monroe ny')]
